In [5]:
from datasets import load_dataset
import pandas as pd
from nltk import pos_tag
import nltk

## Word Dictionaries
- no frequency check

### English

In [80]:
## English
# nltk.download('averaged_perceptron_tagger_eng', download_dir='/work/hyujang/miniconda3/envs/thesis/nltk_data') # for en-pos tagging
nltk.download('wordnet', download_dir='/work/hyujang/miniconda3/envs/thesis/nltk_data') # en vocabulary
# en_ds = load_dataset("manu/project_gutenberg", split="en", streaming=True)
# en_ds = load_dataset("deepmind/pg19", split=None, streaming=True, trust_remote_code=True)
# en_ds = load_dataset("cambridge-climb/BabyLM", split=None, streaming=True, trust_remote_code=True) # smaller size
# en_ds = load_dataset("marksverdhei/wordnet-definitions-en-2021")
# en_df = en_ds['train'].to_pandas()
# data_list = list(en_ds.take(100))

from nltk.corpus import wordnet
noun_synsets = list(wordnet.all_synsets(pos=wordnet.NOUN))
en_nouns = set(lemma.name() for synset in noun_synsets for lemma in synset.lemmas() if "_" not in lemma.name())
print(len(en_nouns))

[nltk_data] Downloading package wordnet to
[nltk_data]     /work/hyujang/miniconda3/envs/thesis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


58690


### Korean

In [ ]:
## Korean
# ds = load_dataset("wicho/kor_3i4k")
# ds = load_dataset("binjang/NIKL-korean-english-dictionary")
import sys
# sys.setdefaultencoding("utf-8")
ds = load_dataset("hyunwoongko/korean-word-dict")
ko_dict = ds['train'].to_pandas()
# ds = load_dataset("hac541309/woori_spring_dict")
# df = ds['train'].to_pandas()
# df_split = df['text'].str.split(", ", expand=True) # takes too long

from konlpy.tag import Okt # TODO: Mecab
import unicodedata
from tqdm import tqdm
tqdm.pandas()
okt = Okt()
ko_dict['word'] = ko_dict['word'].progress_apply(lambda x: unicodedata.normalize("NFC", x)) # ㄴㅐ -> 내
ko_dict['pos_okt'] = ko_dict['word'].progress_apply(lambda x: okt.pos(x))


In [ ]:
ko_dict_filtered = ko_dict[
    (ko_dict['pos_okt'].apply(len) == 1) & 
    (ko_dict['pos_okt'].apply(lambda x: x[0][1] == 'Noun')) &
    (ko_dict['word'].apply(lambda x: len(x) > 1)) &
    (ko_dict['freq'].apply(lambda x: x > 0))
]

ko_dict_filtered = (
    ko_dict_filtered.groupby('word', as_index=False)
    .agg({
        'freq': 'sum',  # Sum up the 'freq' column for duplicates
        **{col: 'first' for col in ko_dict_filtered.columns if col not in ['word', 'freq']}  # Keep the first occurrence for other columns
    })
)

In [88]:
ko_dict_filtered = pd.read_csv("../data/korean_dict-hyunwoongko_korean-word-dict.csv")
ko_dict_filtered

,word,freq,morphs_okt,pos_okt
0,우리,18207,['우리'],"[('우리', 'Noun')]"
1,무슨,16460,['무슨'],"[('무슨', 'Noun')]"
2,그냥,15446,['그냥'],"[('그냥', 'Noun')]"
3,정말,14968,['정말'],"[('정말', 'Noun')]"
4,여기,13454,['여기'],"[('여기', 'Noun')]"
...,...,...,...,...
46144,크누센,1,['크누센'],"[('크누센', 'Noun')]"
46145,랜덜,1,['랜덜'],"[('랜덜', 'Noun')]"
46146,크라블,1,['크라블'],"[('크라블', 'Noun')]"
46147,오염수,1,['오염수'],"[('오염수', 'Noun')]"


In [6]:
df = pd.read_csv("/home/hyujang/multilingual-inner-lexicon/data/r1_dataset_Babel-9B-Chat_Korean.csv")
df[df['label']=="realword"].sort_values(by='freq', ascending=False)

,word,tokens,token_num,freq,freq_quantile,label
395,교수,"['êµĲ', 'ìĪĺ']",2,1740.0,0.0,realword
349,버스,"['ë²Ħ', 'ìĬ¤']",2,1001.0,0.0,realword
472,인천,"['ìĿ¸', 'ì²ľ']",2,839.0,0.0,realword
304,사단,"['ìĤ¬', 'ëĭ¨']",2,792.0,0.0,realword
418,요청,"['ìļĶ', 'ì²Ń']",2,579.0,0.0,realword
...,...,...,...,...,...,...
720,채웅석,"['ì±Ħ', 'ìĽħ', 'ìĦĿ']",3,1.0,0.0,realword
343,간계,"['ê°Ħ', 'ê³Ħ']",2,1.0,0.0,realword
344,셔벗,"['ìħĶ', 'ë²Ĺ']",2,1.0,0.0,realword
346,건설역,"['ê±´ìĦ¤', 'ìĹŃ']",2,1.0,0.0,realword


### German

In [ ]:
## German
# german_synsets = list(wordnet.all_synsets(lang="deu"))  # 'deu' is the ISO 639-3 code for German -> empty list

from xml.etree import ElementTree as ET
from lxml import etree
import re

def get_wordnet_lexicon_local(wnfile):
     loc_wn = open(wnfile,"r",encoding="utf-8")
     wntree = ET.parse(loc_wn)
     wnroot = wntree.getroot()
     lexicon = wnroot.find('Lexicon')
     return lexicon
 
def extract_all_nouns(wordnet_file):
    lexicon = get_wordnet_lexicon_local(wordnet_file)
    nouns = []
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        pos = lemma.attrib.get('partOfSpeech')
        if pos == 'n':
            nouns.append(lemma_value)
    return nouns
 
wordnet_file = '/work/hyujang/miniconda3/envs/thesis/lib/python3.12/site-packages/odenet/wordnet/deWordNet.xml'
nouns = extract_all_nouns(wordnet_file)
print(len(nouns))

## Wiki

In [ ]:
from kiwipiepy import Kiwi
# ko_noun_freq_df = pd.read_csv("../data/ko_wiki_noun_frequencies.csv")
# df = load_wikipedia_data(lang="ko")

kiwi = Kiwi()
# l = kiwi.tokenize(df['text'][0])
kiwi.tokenize("안녕하세요. 저는 한국어를 공부하고 있습니다.")

[Token(form='안녕', tag='NNG', start=0, len=2),
 Token(form='하', tag='XSA', start=2, len=1),
 Token(form='세요', tag='EF', start=3, len=2),
 Token(form='.', tag='SF', start=5, len=1),
 Token(form='저', tag='NP', start=7, len=1),
 Token(form='는', tag='JX', start=8, len=1),
 Token(form='한국어', tag='NNP', start=10, len=3),
 Token(form='를', tag='JKO', start=13, len=1),
 Token(form='공부', tag='NNG', start=15, len=2),
 Token(form='하', tag='XSV', start=17, len=1),
 Token(form='고', tag='EC', start=18, len=1),
 Token(form='있', tag='VX', start=20, len=1),
 Token(form='습니다', tag='EF', start=21, len=3),
 Token(form='.', tag='SF', start=24, len=1)]

In [ ]:
from datasets import load_dataset
import spacy
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import os
from nltk.stem import WordNetLemmatizer


def load_wikipedia_data(lang, sample_size=20000):
    wiki = load_dataset("wikimedia/wikipedia", f"20231101.{lang}", split="train", columns=["text"])
    return wiki.shuffle(seed=2025).select(range(sample_size)).to_pandas()

def extract_nouns_with_frequency(text, lang, lemmatizer=None, nlp=None, kiwi=None):
    if lang == "en":
        """
        NN = Singular or mass noun
        NNS = Plural noun
        NNP = Singular proper noun (capitalized names)
        NNPS = Plural proper noun (capitalized plural names)
        """
        # doc = nlp(text)
        # nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        nouns = [lemmatizer.lemmatize(word.lower()) for word, tag in tagged if tag in ['NN', 'NNS']]
    elif lang == "de":
        doc = nlp(text)
        nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    elif lang == "ko":
        # nouns = okt.nouns(text)
        doc = kiwi.tokenize(text)
        nouns = [token.form for token in doc if token.tag=="NNG"]
    else:
        raise ValueError("Unsupported language")
    return Counter(nouns)

def process_wikipedia_nouns(lang):
    global df
    
    output_file = f"../data/{lang}_wiki_noun_frequencies_lemmatized.csv"
    if os.path.exists(output_file):
        print(f"Output file '{output_file}' already exists. Skipping processing.")
        return  # Stop the function if the file exists

    print(f"Loading Wikipedia data for language: {lang}")
    df = load_wikipedia_data(lang)
    
    if lang == "en":
        print("Extracting nouns using nltk for English...")
        lemmatizer = WordNetLemmatizer()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, lemmatizer=lemmatizer))

    elif lang == "de":
        print(f"Extracting nouns using spaCy model for {lang}...")
        nlp = spacy.load(f"{lang}_core_news_sm")
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, nlp=nlp))

    elif lang == "ko":
        print(f"Extracting nouns using Okt tokenizer for {lang}...")
        # okt = Okt()
        kiwi = Kiwi()
        tqdm.pandas(desc="Processing text")
        df["noun_frequencies"] = df["text"].progress_apply(lambda text: extract_nouns_with_frequency(text, lang, kiwi=kiwi))
    
    print("Summing noun frequencies...")
    combined_noun_frequencies = sum((Counter(freq_dict) for freq_dict in tqdm(df["noun_frequencies"], desc="Summing")), Counter())
    noun_frequencies_df = pd.DataFrame.from_dict(combined_noun_frequencies, orient="index", columns=["frequency"])
    noun_frequencies_df.sort_values(by="frequency", ascending=False, inplace=True)
    noun_frequencies_df.reset_index(inplace=True)
    noun_frequencies_df.columns = ["word", "freq"]
    # output_file = f"../data/{lang}_wiki_noun_frequencies.csv"
    noun_frequencies_df.to_csv(output_file, index=False)
    print(f"Saved noun frequencies to {output_file}")


process_wikipedia_nouns("en")
# process_wikipedia_nouns("de")
# process_wikipedia_nouns("ko")


Loading Wikipedia data for language: ko
Extracting nouns using Okt tokenizer for ko...


Processing text: 100%|██████████| 20000/20000 [11:49<00:00, 28.18it/s] 


Summing noun frequencies...


Summing: 100%|██████████| 20000/20000 [06:42<00:00, 49.73it/s]


Saved noun frequencies to ../data/ko_wiki_noun_frequencies_kiwi.csv
